In [ ]:
import pandas as pd
import numpy as np
import vector
import os
from xgboost import XGBClassifier

import HH4b.utils as utils
from HH4b.utils import ShapeVar
import HH4b.plotting as plotting
import HH4b.postprocessing as postprocessing
from HH4b.postprocessing import Region

import hist
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
samples = {
    "2022EE": {
        "ttbar": [
            "TTto4Q",
            "TTtoLNu2Q",
            "TTto2L2Nu",
        ],
        "data": [
            "JetMET_Run2022E",
            "JetMET_Run2022F",
            "JetMET_Run2022G",
        ],
        "qcd": [
            "QCD_HT-200to400",
            "QCD_HT-400to600",
            "QCD_HT-600to800",
            "QCD_HT-800to1000",
            "QCD_HT-1000to1200",
            "QCD_HT-1200to1500",
            "QCD_HT-1500to2000",
            "QCD_HT-2000",
        ],
        "diboson": [
            "WW",
            "ZZ",
            "WZ",
        ],
        "vjets": [
            "Wto2Q-3Jets_HT-200to400",
            "Wto2Q-3Jets_HT-400to600",
            "Wto2Q-3Jets_HT-600to800",
            "Wto2Q-3Jets_HT-800",
            "Zto2Q-4Jets_HT-200to400",
            "Zto2Q-4Jets_HT-400to600",
            "Zto2Q-4Jets_HT-600to800",
            "Zto2Q-4Jets_HT-800",
        ],  
        "vjetslnu": [
            "WtoLNu-2Jets",
        ]
    },
}

MAIN_DIR = "../../../"
dir_name = "Mar26_v12_had-tt"
path_to_dir = f"{MAIN_DIR}/../data/skimmer/{dir_name}/"
dirs = {path_to_dir: samples}

load_columns = [
    ("weight", 1),
    ("MET_pt", 1),
    ("nFatJets", 1),
    ("bbFatJetPt", 2),
    ("bbFatJetEta", 2),
    ("bbFatJetPhi", 2),
    ("bbFatJetMsd", 2),
    ("bbFatJetPNetMass", 2),
    ("bbFatJetPNetXbb", 2),
    ("bbFatJetTau3OverTau2", 2),
    ("bbFatJetPNetQCD0HF", 2),
    ("bbFatJetPNetQCD1HF", 2),
    ("bbFatJetPNetQCD2HF", 2),
]


events_dict = {}
years = ["2022EE"]
for year in years:
    events_dict[year] = {}
    for input_dir, samples in dirs.items():
        events_dict[year] = {
            **events_dict[year],
            **utils.load_samples(
                input_dir,
                samples[year],
                year,
                filters=None,
                columns_mc=utils.format_columns(load_columns),
            ),
        }


In [ ]:
list(events_dict["2022EE"]["ttbar"].columns)

In [ ]:
import importlib
import sys

sys.path.append("../boosted/bdt_trainings_run3/")
bdt_model = XGBClassifier()
model_name = "v0_msd30"
bdt_model.load_model(
    fname=f"../boosted/bdt_trainings_run3/{model_name}/trained_bdt.model"
)
# get function
make_bdt_dataframe = importlib.import_module(f"{model_name}")

events_dict_bdt = {}
for year in years:
    events_dict_bdt[year] = {}
    for key in events_dict[year].keys():
        events = events_dict[year][key]

        # implement cuts from BDT training
        #h1msd = events["bbFatJetMsd"].to_numpy()[:, 0]
        #h2msd = events["bbFatJetMsd"].to_numpy()[:, 1]
        #events = events[(h1msd > 30) & (h2msd > 30) & (h1msd < 250) & (h2msd < 250) & (h1pt>300) & (h2pt>300)]

        # perform inference
        df_events = make_bdt_dataframe.bdt_dataframe(events)
        bdt_score = bdt_model.predict_proba(df_events)[:, 1]
        events_dict_bdt[year][key] = df_events

        # add variables
        events_dict_bdt[year][key]["bdt_score"] = bdt_score
        events_dict_bdt[year][key]["weight"] = events["weight"].to_numpy()[:, 0]
        events_dict_bdt[year][key]["H1Pt"] = events["bbFatJetPt"].to_numpy()[:, 0]
        events_dict_bdt[year][key]["H2Pt"] = events["bbFatJetPt"].to_numpy()[:, 1]
        events_dict_bdt[year][key]["H1Msd"] = events["bbFatJetMsd"].to_numpy()[:, 0]
        events_dict_bdt[year][key]["H2Msd"] = events["bbFatJetMsd"].to_numpy()[:, 1]
        events_dict_bdt[year][key]["H2Xbb"] = events["bbFatJetPNetXbb"].to_numpy()[:, 1]
        events_dict_bdt[year][key]["H2PNetMass"] = events["bbFatJetPNetMass"].to_numpy()[:, 1]

        h1 = vector.array(
            {
                "pt": events["bbFatJetPt"].to_numpy()[:, 0],
                "phi": events["bbFatJetPhi"].to_numpy()[:, 0],
                "eta": events["bbFatJetEta"].to_numpy()[:, 0],
                "M": events["bbFatJetPNetMass"].to_numpy()[:, 0],
            }
        )
        h2 = vector.array(
            {
                "pt": events["bbFatJetPt"].to_numpy()[:, 1],
                "phi": events["bbFatJetPhi"].to_numpy()[:, 1],
                "eta": events["bbFatJetEta"].to_numpy()[:, 1],
                "M": events["bbFatJetPNetMass"].to_numpy()[:, 1],
            }
        )
        hh = h1 + h2
        events_dict_bdt[year][key]["HHPt"] = hh.pt

In [ ]:
events_dict_bdt["2022EE"].keys()

In [ ]:
control_plot_vars = [
    ShapeVar(var="HHPt", label=r"$p_{T}^{1,2}$ (GeV)", bins=[0,50,100,150,200,300,400,500,600,700,800,900,1000], reg=False),
    ShapeVar(var="H2Msd", label=r"$m_{SD}^{2}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Msd", label=r"$m_{SD}^{1}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Mass", label=r"$m_{reg}^{1}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="H2PNetMass", label=r"$m_{reg}^{2}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="bdt_score", label=r"BDT score", bins=[30, 0, 1]),
    ShapeVar(var="H1Xbb", label=r"Xbb$^{1}$", bins=[30, 0.8, 1]),
    ShapeVar(var="H2Xbb", label=r"Xbb$^{2}$", bins=[30, 0.8, 1]),
    ShapeVar(var="H1T32", label=r"$\tau_{32}^{1}$", bins=[30, 0, 1]),
    ShapeVar(var="H2T32", label=r"$\tau_{32}^{2}$", bins=[30, 0, 1]),
    ShapeVar(var="H2Pt", label=r"$p_{T}^{2}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="H1Pt", label=r"$p_{T}^{1}$ (GeV)", bins=[30, 0, 600]),
]

ylims = {
    "2022": 5e4,
    "2022EE": 4e3,
    "2023-pre-BPix": 4e5,
}

events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_bdt[year].keys():
        events = events_dict_bdt[year][key]
        events = events[(events["H1Xbb"] > 0.9) & (events["H1Msd"] > 150) & (events["H1Msd"] < 200) & (events["H1Pt"] > 300) & (events["H2Pt"] > 300)]
        events_plot[year][key] = events
        
print(events_plot["2022EE"]["ttbar"].columns)

tag = "tt-had-xbb09"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )

In [ ]:
control_plot_vars = [
    ShapeVar(var="H2Msd", label=r"$m_{SD}^{2}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Msd", label=r"$m_{SD}^{1}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Mass", label=r"$m_{reg}^{1}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="H2PNetMass", label=r"$m_{reg}^{2}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="bdt_score", label=r"BDT score", bins=[30, 0, 1]),
    ShapeVar(var="H1Xbb", label=r"Xbb$^{1}$", bins=[30, 0.1, 1]),
    ShapeVar(var="H2Xbb", label=r"Xbb$^{2}$", bins=[30, 0.1, 1]),
    ShapeVar(var="H1T32", label=r"$\tau_{32}^{1}$", bins=[30, 0, 1]),
    ShapeVar(var="H2T32", label=r"$\tau_{32}^{2}$", bins=[30, 0, 1]),
    ShapeVar(var="H2Pt", label=r"$p_{T}^{2}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="H1Pt", label=r"$p_{T}^{1}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="HHPt", label=r"$p_{T}^{1,2}$ (GeV)", bins=[0,50,100,150,200,300,400,500,600,700,800,900,1000], reg=False),
]

ylims = {
    "2022": 5e4,
    #"2022EE": 4e3,
    "2022EE": 4e5,
    "2023-pre-BPix": 4e5,
}

events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_bdt[year].keys():
        events = events_dict_bdt[year][key]
        events = events[(events["H1Xbb"] > 0.1) & (events["H2Xbb"] > 0.1) & (events["H1Msd"] > 50) & (events["H1Pt"] > 300) & (events["H2Pt"] > 300)]
        #events = events[(events["H1Xbb"] > 0.1) & (events["H2Xbb"] > 0.1) & (events["H1Msd"] > 50) & (events["H1Pt"] > 300) & (events["H2Pt"] > 300) & (events["H1T32"] < 0.46) & (events["H2T32"] < 0.46)]
        events_plot[year][key] = events
        
print(events_plot["2022EE"]["ttbar"].columns)

tag = "tt-had"
#tag = "tt-had-t32"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )

control_plot_vars = [
    ShapeVar(var="bdt_score", label=r"BDT score", bins=[0, 0.005, 0.01, 0.5, 0.9, 1], reg=False),
]

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}_logx",
            sortyield=True,
            show=False,
            log=True,
            logx=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )




In [ ]:
samples = {
    "2022EE": {
        "ttbar": [
            "TTto4Q",
            "TTtoLNu2Q",
            "TTto2L2Nu",
        ],
        "data": [
            "EGamma_Run2022E",
            "EGamma_Run2022F",
            "EGamma_Run2022G",
            "Muon_Run2022E",
            "Muon_Run2022F",
            "Muon_Run2022G",
        ],
        "qcd": [
            "QCD_HT-200to400",
            "QCD_HT-400to600",
            "QCD_HT-600to800",
            "QCD_HT-800to1000",
            "QCD_HT-1000to1200",
            "QCD_HT-1200to1500",
            "QCD_HT-1500to2000",
            "QCD_HT-2000",
        ],
        "diboson": [
            "WW",
            "ZZ",
            "WZ",
        ],
        "vjets": [
            "Wto2Q-3Jets_HT-200to400",
            "Wto2Q-3Jets_HT-400to600",
            "Wto2Q-3Jets_HT-600to800",
            "Wto2Q-3Jets_HT-800",
            "Zto2Q-4Jets_HT-200to400",
            "Zto2Q-4Jets_HT-400to600",
            "Zto2Q-4Jets_HT-600to800",
            "Zto2Q-4Jets_HT-800",
        ],
        "vjetslnu": [
            "WtoLNu-4Jets",
        ],
    },
}

MAIN_DIR = "../../../"
dir_name = "24Mar01_v12_semilep-tt"
path_to_dir = f"{MAIN_DIR}/../data/skimmer/{dir_name}/"
dirs = {path_to_dir: samples}

load_columns = [
    ("weight", 1),
    ("MET_pt", 1),
    ("nFatJets", 1),
    ("bbFatJetPt", 2),
    ("bbFatJetEta", 2),
    ("bbFatJetPhi", 2),
    ("bbFatJetMsd", 2),
    ("bbFatJetPNetMass", 2),
    ("bbFatJetPNetXbb", 2),
    ("bbFatJetTau3OverTau2", 2),
    ("ak8FatJetPt", 2),
    ("ak8FatJetEta", 2),
    ("ak8FatJetPhi", 2),
    ("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
    ("ak8FatJetPNetXbb", 2),
    ("ak8FatJetTau3OverTau2", 2),
    #("bbFatJetPNetQCD0HF", 2),
    #("bbFatJetPNetQCD1HF", 2),
    #("bbFatJetPNetQCD2HF", 2),
]

pt_cut = 300
pt_veto = 200
msd_cut = 50
filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", pt_cut),
        ("('ak8FatJetPt', '1')", "<", pt_veto),
        ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        ("('ak8FatJetMsd', '1')", "<", msd_cut),
    ],
]


events_dict = {}
years = ["2022EE"]
for year in years:
    events_dict[year] = {}
    for input_dir, samples in dirs.items():
        events_dict[year] = {
            **events_dict[year],
            **utils.load_samples(
                input_dir,
                samples[year],
                year,
                filters=filters,
                columns_mc=utils.format_columns(load_columns),
            ),
        }


In [ ]:
events_dict_tt = {}
for year in years:
    events_dict_tt[year] = {}
    for key in events_dict[year].keys():
        events = events_dict[year][key]
        events_dict_tt[year][key] = pd.DataFrame()
        events_dict_tt[year][key]["weight"] = events["weight"].to_numpy()[:, 0]

        jet_str = "bb"
        #jet_str = "ak8"
        events_dict_tt[year][key]["H1Msd"] = events[f"{jet_str}FatJetMsd"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Msd"] = events[f"{jet_str}FatJetMsd"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1Pt"] = events[f"{jet_str}FatJetPt"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Pt"] = events[f"{jet_str}FatJetPt"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1T32"] = events[f"{jet_str}FatJetTau3OverTau2"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2T32"] = events[f"{jet_str}FatJetTau3OverTau2"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1Xbb"] = events[f"{jet_str}FatJetPNetXbb"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Xbb"] = events[f"{jet_str}FatJetPNetXbb"].to_numpy()[:, 1]

        h1 = vector.array(
            {
                "pt": events[f"{jet_str}FatJetPt"].to_numpy()[:, 0],
                "phi": events[f"{jet_str}FatJetPhi"].to_numpy()[:, 0],
                "eta": events[f"{jet_str}FatJetEta"].to_numpy()[:, 0],
                "M": events[f"{jet_str}FatJetPNetMass"].to_numpy()[:, 0],
            }
        )
        h2 = vector.array(
            {
                "pt": events[f"{jet_str}FatJetPt"].to_numpy()[:, 1],
                "phi": events[f"{jet_str}FatJetPhi"].to_numpy()[:, 1],
                "eta": events[f"{jet_str}FatJetEta"].to_numpy()[:, 1],
                "M": events[f"{jet_str}FatJetPNetMass"].to_numpy()[:, 1],
            }
        )
        hh = h1 + h2
        events_dict_tt[year][key]["HHPt"] = hh.pt

In [ ]:
events_dict_tt["2022EE"]["vjetslnu"]

In [ ]:
control_plot_vars = [
    ShapeVar(var="H2Msd", label=r"$m_{SD}^{2}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Msd", label=r"$m_{SD}^{1}$ (GeV)", bins=[30, 30, 300]),
    #ShapeVar(var="H1Mass", label=r"$m_{reg}^{1}$ (GeV)", bins=[30, 0, 300]),
    #ShapeVar(var="H2PNetMass", label=r"$m_{reg}^{2}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="H1Xbb", label=r"Xbb$^{1}$", bins=[30, 0., 1]),
    ShapeVar(var="H2Xbb", label=r"Xbb$^{2}$", bins=[30, 0., 1]),
    ShapeVar(var="H1T32", label=r"$\tau_{32}^{1}$", bins=[30, 0, 1]),
    ShapeVar(var="H2T32", label=r"$\tau_{32}^{2}$", bins=[30, 0, 1]),
    ShapeVar(var="H2Pt", label=r"$p_{T}^{2}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="H1Pt", label=r"$p_{T}^{1}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="HHPt", label=r"$p_{T}^{1,2}$ (GeV)", bins=[0,50,100,150,200,300,400,500,600,700,800,900,1000], reg=False),
]

ylims = {
    "2022": 5e4,
    "2022EE": 4e5,
    "2023-pre-BPix": 4e5,
}

events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_tt[year].keys():
        events = events_dict_tt[year][key]
        #events = events[(events["H1Msd"] > 140)]
        #events = events[(events["H1Msd"] > 140) & (events["H1T32"]<0.46)]
        events_plot[year][key] = events
        
print(events_plot["2022EE"]["ttbar"].columns)

tag = "tt-semi-lep"
#tag = "tt-semi-lep-msd140"
#tag = "tt-semi-lep-msd140-t32"
#tag = "tt-semi-lep-bypt"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )

control_plot_vars = [
    ShapeVar(var="H1Xbb", label=r"Xbb$^{1}$", bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.92, 0.94, 0.96, 1], reg=False),
]

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}_logx",
            sortyield=True,
            show=False,
            log=True,
            # logx=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )